In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Type_Code'].tolist()
list_part_key = df['Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')).strip() for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Key': part_key,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Key': part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：256

9679837  <->  [ok] - 剩余数量：246
5413263  <->  [ok] - 剩余数量：245
5413251  <->  [ok] - 剩余数量：244
9679845  <->  [ok] - 剩余数量：243
9679841  <->  [ok] - 剩余数量：242
9679849  <->  [ok] - 剩余数量：241
5413266  <->  [ok] - 剩余数量：240
5413260  <->  [ok] - 剩余数量：239
13957601  <->  [ok] - 剩余数量：238
5414493  <->  [ok] - 剩余数量：237
12770421  <->  [ok] - 剩余数量：236
5414496  <->  [ok] - 剩余数量：235
2415158  <->  [ok] - 剩余数量：234
7988428  <->  [ok] - 剩余数量：233
5413257  <->  [ok] - 剩余数量：232
5413254  <->  [ok] - 剩余数量：231
5414499  <->  [ok] - 剩余数量：230
8769172  <->  [ok] - 剩余数量：229
8148876  <->  [ok] - 剩余数量：228
4691220  <->  [ok] - 剩余数量：227
4632954  <->  [ok] - 剩余数量：226
6588520  <->  [ok] - 剩余数量：225
4693146  <->  [ok] - 剩余数量：224
8972592  <->  [ok] - 剩余数量：223
5414505  <->  [ok] - 剩余数量：222
5414502  <->  [ok] - 剩余数量：221
5414511  <->  [ok] - 剩余数量：220
5414508  <->  [ok] - 剩余数量：219
3768753  <->  [ok] - 剩余数量：218
3778695  <->  [ok] - 剩余数量：217
5414520  <->  [ok] - 剩余数量：216
5414514  <->  [ok] - 剩余数量：215
12770893  <->  [ok] - 剩余数量：21